In [1]:
import os
from qonscious.adapters import IonQBackendAdapter
api_key = os.getenv("IONQ_API_KEY")
adapter = IonQBackendAdapter.aria_simulator(api_key)

/home/tiagorestucha/Desktop/qonscious/.venv/lib/python3.11/site-packages/qiskit_ionq/ionq_backend.py:127: IonQTranspileLevelWarning: Transpiler default optimization_level=2. IonQ (QIS) recommends 0-1 to avoid aggressive re-synthesis; use transpile(..., optimization_level=1).
  warn_bad_transpile_level()


In [2]:
from qiskit import QuantumCircuit


# A helper function to create our phi_plus circuit.
def build_phi_plus_circuit() :
    phi_plus = QuantumCircuit(2)
    phi_plus.h(0)
    phi_plus.cx(0, 1)
    phi_plus.measure_all()
    return phi_plus

In [3]:
import json
import math
from typing import Any, Dict
import matplotlib.pyplot as plt

def on_pass(backend_adapter, figureOfMeritResults):
    print("\n--- Checks PASSED ---")
    props = figureOfMeritResults[0].get('properties', {})
    print("\n--- Calculated Properties ---")
    print(json.dumps(props, indent=2))
    print(f"\nNumber of Qubits: {getattr(backend_adapter, 'n_qubits', 'N/A')}")
    run_result = backend_adapter.run(build_phi_plus_circuit(), shots=2048)
    return run_result

def on_fail(backend_adapter, figureOfMeritResults):
    print("\n--- Checks FAILED ---")
    props = figureOfMeritResults[0].get('properties', {})
    print("\n--- Calculated Properties ---")
    print(json.dumps(props, indent=2))
    print(f"\nNumber of Qubits: {getattr(backend_adapter, 'n_qubits', 'N/A')}")
    return None

In [4]:
from qonscious.checks import MeritComplianceCheck
from qonscious.foms import BackendNoiseFoM
MIN_OVERALL_NOISE = 0.985
MIN_TWO_QUBIT_FIDELITY = 0.985
MIN_READOUT_FIDELITY = 0.970


def requires_noise_quality():
    """Return a predicate that validates the backend against the configured thresholds."""
    def predicate(result):
        props = result["properties"]
        return (
            props["overall_noise_score"] >= MIN_OVERALL_NOISE
            and props["two_qubit_gate_fidelity"] >= MIN_TWO_QUBIT_FIDELITY
            and props["readout_fidelity"] >= MIN_READOUT_FIDELITY
               
        )
    return predicate
check = MeritComplianceCheck(
    figure_of_merit=BackendNoiseFoM(),
    decision_function=requires_noise_quality()
)

In [5]:
from qonscious.actions import QonsciousCallable
from qonscious import run_conditionally

qonscious_result = run_conditionally(
    backend_adapter=adapter,
    checks=[check],
    on_pass=QonsciousCallable(on_pass),
    on_fail=QonsciousCallable(on_fail),
)


--- Checks PASSED ---

--- Calculated Properties ---
{
  "backend_qubits": 29,
  "backend_name": "IonQBackendAdapter",
  "T1": 4.9999999999999996e-05,
  "T2": 7e-05,
  "overall_noise_score": 0.9993755936195775,
  "single_qubit_gate_fidelity": 1.0,
  "two_qubit_gate_fidelity": 1.0,
  "readout_fidelity": 1.0,
  "coherence_factor": 0.9975047128046468,
  "single_qubit_gate_duration_seconds": 5.0000000000000004e-08,
  "two_qubit_gate_duration_seconds": 3.0000000000000004e-07,
  "thermal_population_fraction": 0.0
}

Number of Qubits: 29


IonQCredentialsError: IonQCredentialsError('Credentials `token` may not be None!')